# Task Settings

In [1]:
import os
os.environ['OPENAI_API_KEY']=""


from utils import *
from prompts import Prompts, TASK_LABLES, TAGS

dataset_name = 'pandemic'
test_model = "experimental:gpt-4o-mini"
eval_model = "gpt-4o" 
iteration = 1
date = '0919'
total_steps=5   
epoch=1
batch_size=3

# Initialize

In [2]:
cm_labels = TASK_LABLES[dataset_name]
tags = TAGS[dataset_name]
CAUSAL_SYSTEM = Prompts[dataset_name]['CAUSAL_SYSTEM']
CAUSAL_SYSTEM_CONSTRAINT = Prompts[dataset_name]['CAUSAL_SYSTEM_CONSTRAINT']
SYSTEM = Prompts[dataset_name]['SYSTEM']

llm_api_eval = tg.get_engine(engine_name=eval_model)
llm_api_test = tg.get_engine(engine_name=test_model, cache=False)
tg.set_backward_engine(llm_api_eval, override=True)

train_set, val_set, test_set_ori, eval_fn = load_task(dataset_name, evaluation_api=llm_api_eval, prompt_col="organized_prompt")
train_loader = tg.tasks.DataLoader(train_set, batch_size=batch_size, shuffle=True)

# Add Tag
col = "organized_prompt" if dataset_name == 'swiss' else "prompt"
train_set.data[col] = train_set.data[col].apply(lambda x: f"{tags[0]}{x}{tags[1]}")
val_set.data[col] = val_set.data[col].apply(lambda x: f"{tags[0]}{x}{tags[1]}")
test_set_ori.data[col] = test_set_ori.data[col].apply(lambda x: f"{tags[0]}{x}{tags[1]}")
print("Train/Val/Test Set Lengths: ", len(train_set), len(val_set), len(test_set_ori))

Train/Val/Test Set Lengths:  100 100 100


In [3]:
from copy import deepcopy

test_set = deepcopy(test_set_ori)
system_prompt, causal_prompt, model, causal_model, optimizer, optimizer_causal = init(SYSTEM, CAUSAL_SYSTEM, llm_api_test, llm_api_eval, CAUSAL_SYSTEM_CONSTRAINT)
results = {"test_f1": [], "prompt": [], "validation_f1": [], 'system_prompt':[], 'causal_prompt': []}

In [4]:
acc, t_rs, t_ys = eval_dataset(test_set, eval_fn, model,iters=1)
test_res = evaluate_metrics(t_rs, t_ys, cm_labels)

print(test_res)

Accuracy: 0.3300: 100%|██████████| 100/100 [00:07<00:00, 12.82it/s]

{'accuracy': 0.33, 'precision': 0.3681077694235589, 'f1': 0.32949282211156405, 'cm': array([[ 5,  9,  2,  3,  0],
       [ 2,  6,  6,  3,  1],
       [ 1,  5,  5,  5,  3],
       [ 0,  1,  5,  3, 10],
       [ 0,  0,  1,  4, 14]])}


In [4]:
acc, t_rs, t_ys = eval_dataset(test_set, eval_fn, model,iters=1)
test_res = evaluate_metrics(t_rs, t_ys, cm_labels)

print(test_res)

Accuracy: 0.3600: 100%|██████████| 100/100 [00:10<00:00,  9.38it/s]

{'accuracy': 0.36, 'precision': 0.3622222222222222, 'f1': 0.3475909285408643, 'cm': array([[ 5,  9,  3,  3,  0],
       [ 3,  5,  7,  2,  2],
       [ 2,  2,  7,  6,  3],
       [ 0,  2,  4,  4, 10],
       [ 0,  0,  0,  5, 15]])}


In [6]:
acc, t_rs, t_ys = eval_dataset(test_set, eval_fn, model,iters=1)
test_res = evaluate_metrics(t_rs, t_ys, cm_labels)

print(test_res)

Accuracy: 0.3800: 100%|██████████| 100/100 [00:54<00:00,  1.82it/s]

{'accuracy': 0.38, 'precision': 0.3998739495798319, 'f1': 0.36617022617022615, 'cm': array([[ 3,  8,  3,  4,  0],
       [ 2,  9,  3,  4,  1],
       [ 1,  4,  6,  5,  2],
       [ 0,  3,  4,  4,  9],
       [ 0,  0,  1,  3, 16]])}


In [4]:
import time
import copy
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

NUM_WORKERS = 1

def run_one_worker(worker_id: int):

    local_test_set = copy.deepcopy(test_set)
    local_test_set_ori = copy.deepcopy(test_set_ori)
    local_val_set = copy.deepcopy(val_set)
    local_train_loader = train_loader

    val_performance = -float('inf')
    test_performance = -float('inf')
    final_results = None
    all_val_f1s = []
    all_test_f1s = []

    local_test_set.data[col] = local_test_set_ori.data[col].apply(
        lambda x: f"<!-- {time.time()} (w{worker_id}) -->, {x}"
    )

    for cur_iter in range(iteration):
        print(f"[Worker {worker_id}] [Iteration {cur_iter+1}/{iteration}] begin")
        output_json = (
            f"res/{date}_{dataset_name}_{test_model.split('/')[-1].split(':')[-1]}_"
            f"w{worker_id}_it{cur_iter+1}.json"
        )
        initialize_json_file(output_json)

        system_prompt, causal_prompt, model, causal_model, optimizer, optimizer_causal = init(
            SYSTEM, CAUSAL_SYSTEM, llm_api_test, llm_api_eval, CAUSAL_SYSTEM_CONSTRAINT
        )

        results, test_res, val_res = init_eval(
            local_val_set, local_test_set, eval_fn, model, causal_model,
            system_prompt, causal_prompt, cm_labels, iters=ITERS    
        )

        results = run_training(
            local_train_loader, local_val_set, local_test_set, eval_fn,
            model, causal_model, system_prompt, causal_prompt,
            optimizer, optimizer_causal, results, cm_labels,
            output_json=output_json, epoch=epoch, steps=total_steps, iters=ITERS
        )

        all_val_f1s.append(results['validation_f1'])
        all_test_f1s.append(results['test_f1'])

        cur_val = results['validation_f1'][-1]
        cur_test = results['test_f1'][-1]
        if cur_val > val_performance:
            val_performance = cur_val
            test_performance = cur_test
            final_results = results

        print(f"[Worker {worker_id}] [Iteration {cur_iter+1}] "
              f"val_best={val_performance:.4f}, test_at_best={test_performance:.4f}")

    return {
        'best_test_f1': test_performance,
        'val_f1s': all_val_f1s,
        'test_f1s': all_test_f1s,
        'worker_id': worker_id,
    }


# Running the cell below will incur API usage charges. Refer to our paper for detailed cost

In [5]:
NUM_WORKERS = 3
ITERS = 1
total_steps=5
batch_size=3

EGO_res = []

with ThreadPoolExecutor(max_workers=NUM_WORKERS) as ex:
    futures = [ex.submit(run_one_worker, i) for i in range(NUM_WORKERS)]
    for fut in as_completed(futures):
        res = fut.result()
        EGO_res.append(res['best_test_f1'])
        print(f"[Main] Worker {res['worker_id']} done. Best test_f1={res['best_test_f1']:.4f}")

print("EGO_res (best test F1 per worker):", EGO_res)

[Worker 0] [Iteration 1/1] begin[Worker 1] [Iteration 1/1] begin

[Worker 2] [Iteration 1/1] begin


Accuracy: 0.4700: 100%|██████████| 100/100 [00:16<00:00,  6.19it/s]


SCG_val_f1: 0.47891035970830037, SCG_test_f1:0.37069647287038593


0it [00:00, ?it/s]


Epoch 0, Step 0


...

Skip Test
[Causal Validation] F1: 0.3797, Previous F1: 0.4790
[Causal Validation CM]:
[[ 2  5  8  3  0]
 [ 1  6  6  6  0]
 [ 0  0 10 11  0]
 [ 0  0  1 11  4]
 [ 0  0  1 13  9]]
Skip Test
[Worker 2] [Iteration 1] val_best=0.4790, test_at_best=0.3141
[Main] Worker 2 done. Best test_f1=0.3141


Accuracy: 0.4500: 100%|██████████| 100/100 [00:29<00:00,  3.39it/s]


Skip Test
[Causal Validation] F1: 0.4472, Previous F1: 0.5121
[Causal Validation CM]:
[[ 3 13  0  1  0]
 [ 2 10  4  4  0]
 [ 0  5  8  9  0]
 [ 0  0  2 11  4]
 [ 0  0  2  8 13]]


Accuracy: 0.4100: 100%|██████████| 100/100 [00:32<00:00,  3.08it/s]
5it [16:33, 198.76s/it]

[Test Result] F1: 0.3880
[Worker 1] [Iteration 1] val_best=0.5121, test_at_best=0.3880
[Main] Worker 1 done. Best test_f1=0.3880
EGO_res (best test F1 per worker): [0.394791520401759, 0.3141095077981786, 0.3879919437062294]
